In [1]:
%load_ext autoreload
%autoreload 2

# Importar bibliotecas padrão
import time
from datetime import timedelta
import os

# Carregar variáveis de ambiente
from dotenv import load_dotenv
load_dotenv(os.path.join('config', '.env'))

# Importar bibliotecas para PostgreSQL
import psycopg2

# Importar bibliotecas para Spotify API
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import src.spotify_data as spotify_data

# Importar bibliotecas para PySpark
from pyspark.sql import SparkSession


jana = True
start_time_geral = time.time()

## Configuração de conexão Pyspark

In [2]:
# Configurar SparkSession
spark = SparkSession.builder \
    .appName("PostgreSQL to Spark") \
    .config("spark.jars", "/caminho/para/postgresql-<versao>.jar") \
    .getOrCreate()


In [3]:
if jana:
    db_config = {
        'host': os.getenv('DB_HOST'),
        'port': os.getenv('DB_PORT'),
        'dbname': os.getenv('DB_NAME_JANA'),
        'user': os.getenv('DB_USER'),
        'password': os.getenv('DB_PASSWORD')
    }
else:
    db_config = {
        'host': os.getenv('DB_HOST'),
        'port': os.getenv('DB_PORT'),
        'dbname': os.getenv('DB_NAME'),
        'user': os.getenv('DB_USER'),
        'password': os.getenv('DB_PASSWORD')
    }

# URL de conexão JDBC
jdbc_url = f"jdbc:postgresql://{db_config['host']}:{db_config['port']}/{db_config['dbname']}"

# Propriedades de conexão
connection_properties = {
    "user": db_config['user'],
    "password": db_config['password'],
    "driver": "org.postgresql.Driver"
}

## Configuração de conexão ao Spotify

In [4]:
# Carregar variáveis de ambiente do arquivo .env na pasta config 
if jana:
    client_id = os.getenv('CLIENT_ID_JANA') 
    client_secret = os.getenv('CLIENT_SECRET_JANA')
else:
    client_id = os.getenv('CLIENT_ID') 
    client_secret = os.getenv('CLIENT_SECRET')

# Configuração da autenticação
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Coleta de dados: Banco de dados

In [5]:
start_time = time.time()

In [6]:
df = spark.read.jdbc(url=jdbc_url, table="all_tracks_registry", properties=connection_properties)
df.show(4)


+-------------------+--------------------+---------+------------+--------------+--------------------------+---------------------------------+--------------------------------+--------------------+------------+-----------------+-------------------+------------+----------+-------+-------+-------+-----------------+--------------+
|                 ts|            platform|ms_played|conn_country|       ip_addr|master_metadata_track_name|master_metadata_album_artist_name|master_metadata_album_album_name|   spotify_track_uri|episode_name|episode_show_name|spotify_episode_uri|reason_start|reason_end|shuffle|skipped|offline|offline_timestamp|incognito_mode|
+-------------------+--------------------+---------+------------+--------------+--------------------------+---------------------------------+--------------------------------+--------------------+------------+-----------------+-------------------+------------+----------+-------+-------+-------+-----------------+--------------+
|2021-09-02 12:0

In [7]:
df.count()

37185

In [8]:
df.printSchema()

root
 |-- ts: timestamp (nullable = true)
 |-- platform: string (nullable = true)
 |-- ms_played: integer (nullable = true)
 |-- conn_country: string (nullable = true)
 |-- ip_addr: string (nullable = true)
 |-- master_metadata_track_name: string (nullable = true)
 |-- master_metadata_album_artist_name: string (nullable = true)
 |-- master_metadata_album_album_name: string (nullable = true)
 |-- spotify_track_uri: string (nullable = true)
 |-- episode_name: string (nullable = true)
 |-- episode_show_name: string (nullable = true)
 |-- spotify_episode_uri: string (nullable = true)
 |-- reason_start: string (nullable = true)
 |-- reason_end: string (nullable = true)
 |-- shuffle: boolean (nullable = true)
 |-- skipped: boolean (nullable = true)
 |-- offline: boolean (nullable = true)
 |-- offline_timestamp: timestamp (nullable = true)
 |-- incognito_mode: boolean (nullable = true)



In [9]:
end_time = time.time()
elapsed_time = end_time - start_time
# Converter o tempo decorrido para o formato HH:MM:SS 
print(f"Tempo de processamento: {str(timedelta(seconds=int(elapsed_time)))}")

Tempo de processamento: 0:00:11


## Coleta dados do Spotify e cria novos bancos

In [10]:
start_time = time.time()

In [11]:
# Funções de banco de dados Tracks e Artistics
import src.db_table_track as db_table_track
import src.db_table_artistcs as db_table_artistcs

In [12]:
# Selecionar e coletar os dados distintos 
distinct_uris = df.select('spotify_track_uri').distinct().collect() 
# Transformar em lista de valores simples 
uris_list = [row['spotify_track_uri'] for row in distinct_uris]
print(f'Quantidade de faixas: {len(uris_list)}')

Quantidade de faixas: 6794


In [13]:
print(uris_list[0])

spotify:track:3NBeihcw8SR7rXI0Amd6LH


In [ ]:
n = 0
for track_uri in uris_list:
    print(f'Faixa número:{n}')
    # Primeiro Verifica se o track já existe na tabela, se FALSE, realiza requisição no spotify
    if db_table_track.track_exists(track_uri, conn=psycopg2.connect(**db_config)):
        print(f'n:{n}, TRACK Já existe no banco')
        main_artist_uri = db_table_track.get_main_artist_uri(track_uri, conn=psycopg2.connect(**db_config))

    else:
        # Realiza requisição da faixa e depois inseri no banco
        dados_faixa = spotify_data.obter_dados_faixa(sp = sp, track_uri = track_uri)
        db_table_track.insert_track_data(dados_faixa, conn=psycopg2.connect(**db_config))

        # Coleta de dados do Artista no Spotify
        main_artist_uri = dados_faixa['main_artist_URI']

    if db_table_artistcs.artist_exists(main_artist_uri, conn=psycopg2.connect(**db_config)):
        # Não fazer nada se o artista já existir
        print(f'n:{n}, ARTISTIC Já existe no banco')
        pass
    else:
        # Realiza requisição do artista e depois inseri no banco
        dados_artista = spotify_data.obter_dados_artista(sp=sp, artist_uri = main_artist_uri)
        db_table_artistcs.insert_artist_data(dados_artista, conn=psycopg2.connect(**db_config))
    time.sleep(5)
    n = n + 1  

Faixa número:0
n:0, TRACK Já existe no banco
n:0, ARTISTIC Já existe no banco
Faixa número:1
n:1, TRACK Já existe no banco
n:1, ARTISTIC Já existe no banco
Faixa número:2
n:2, TRACK Já existe no banco
n:2, ARTISTIC Já existe no banco
Faixa número:3
n:3, TRACK Já existe no banco
n:3, ARTISTIC Já existe no banco
Faixa número:4
n:4, TRACK Já existe no banco
n:4, ARTISTIC Já existe no banco
Faixa número:5
Faixa número:6
Faixa número:7
Faixa número:8
Faixa número:9
Faixa número:10
Faixa número:11
Faixa número:12
Faixa número:13
Faixa número:14
Faixa número:15
Faixa número:16
Faixa número:17
Faixa número:18
Faixa número:19
Faixa número:20
Faixa número:21
Faixa número:22
Faixa número:23
Faixa número:24
Faixa número:25
Faixa número:26
Faixa número:27
Faixa número:28
Faixa número:29
Faixa número:30
Faixa número:31
Faixa número:32
Faixa número:33
Faixa número:34
Faixa número:35
Faixa número:36
Faixa número:37
Faixa número:38
Faixa número:39
Faixa número:40
Faixa número:41
Faixa número:42
Faixa n

In [ ]:
end_time = time.time()
elapsed_time = end_time - start_time
# Converter o tempo decorrido para o formato HH:MM:SS 
print(f"Tempo de processamento: {str(timedelta(seconds=int(elapsed_time)))}")


In [ ]:
# import pyspark.sql.functions as F
# df = df.withColumn('ano-mes-dia', F.to_date(F.col("ts")))
# df = df.withColumn("ano-mes", F.concat_ws("-", F.year(df["ts"]), F.month(df["ts"])))
# df = df.withColumn("ano_mes_str", F.date_format(df["ts"], "yyyy-MM")) 
# df = df.withColumn("ano_mes_date", F.to_date(df["ano_mes_str"], "yyyy-MM"))
# df.show(5, truncate=False)